# Sequential Lock

Previous `readers-writer` lock may lead to `writer starvation` problem. A writer process can't acquire a lock as long as at least one reader process which acquired a lock holds it. So, in the situation when contention is high, it will lead to situation when a writer process which wants to acquire a lock will wait for it for a long time.

对于 **写饥饿** 问题，seqlock的解决办法是有一个sequence number。每次写的时候都增加sequence number，其实也就类似于MVCC中的版本号。对于 **读锁** 有两种，第一种是block writer，这种和之前一样。

另一种是不会 block writer，而是在进入critical section之前，先读取sequence number，然后进入critical section，读取完成后，再读取一遍sequence number，跟之前读取的数值比较，如果一样，说明读取中间没有writer操作，就返回。否则继续读取。

--------------------------------------------

```c
/*
 * Reader/writer consistent mechanism without starving writers. This type of
 * lock for data where the reader wants a consistent set of information
 * and is willing to retry if the information changes. There are two types
 * of readers:
 * 1. Sequence readers which never block a writer but they may have to retry
 *    if a writer is in progress by detecting change in sequence number.
 *    Writers do not wait for a sequence reader.
 * 2. Locking readers which will wait if a writer or another locking reader
 *    is in progress. A locking reader in progress will also block a writer
 *    from going forward. Unlike the regular rwlock, the read lock here is
 *    exclusive so that only one locking reader can get it.
 *
 * This is not as cache friendly as brlock. Also, this may not work well
 * for data that contains pointers, because any writer could
 * invalidate a pointer that a reader was following.
 *
 * Expected non-blocking reader usage:
 * 	do {
 *	    seq = read_seqbegin(&foo);
 * 	...
 *      } while (read_seqretry(&foo, seq));
 *
 *
 * On non-SMP the spin locks disappear but the writer still needs
 * to increment the sequence variables because an interrupt routine could
 * change the state of the data.
 *
 * Based on x86_64 vsyscall gettimeofday 
 * by Keith Owens and Andrea Arcangeli
 */
```

---------------------------


```c
/*
 * Version using sequence counter only.
 * This can be used when code has its own mutex protecting the
 * updating starting before the write_seqcountbeqin() and ending
 * after the write_seqcount_end().
 */
typedef struct seqcount {
	unsigned sequence;
} seqcount_t;


typedef struct {
	struct seqcount seqcount;
	spinlock_t lock;
} seqlock_t;

```

1. 


--------------------------